In [1]:
import logging
import sys
from typing import Dict, Tuple
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, roc_curve, precision_score
from sklearn.model_selection import train_test_split

In [2]:
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
def evaluate(classifier, X_test, y_test):
   


    #X_train, X_test, y_train, y_test = train_test_split(data,target)
    #classifier.fit(X_train,y_train)
    y_pred=classifier.predict(X_test)
    probs=classifier.predict_proba(X_test)[:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    pres= precision_score (y_test, y_pred, average='micro')
    f1 = f1_score(y_test, y_pred, average='micro')
    #auc = roc_auc_score(y_test, probs)
    print ("Accurancy: ", acc)
    print ("Precision: ", pres)
    print ("f1: ", f1_score)
    #print ("auc: ", auc)
    
   # fpr, tpr, threshols = roc_curve(y_test, probs)
   # plt.plot(fpr,tpr)
   # plt.xlabel('False positives rate')
   # plt.ylabel('True positives rate')
   # plt.gcf().set_size_inches(8,6)
    return  f1, acc, pres

In [4]:
def saca_info ():
    data=pd.read_csv("../Data/01_raw/Data.csv")
    print ("Antes de limpia")
    print ("Enfermedades: ", data["Enfermedad"].nunique())
    print ("Sintomas: ", data["Sintoma"].nunique())
    print ("Frecuencias: ", data["Frecuencia"].nunique())
    data=data.drop_duplicates()
    data=data.dropna()
   # df.drop(df[(df['precio_unitario'] >400) & (df['precio_unitario'] < 600)].index, inplace=True)
    #sintomas_poca_ocurrencia=serie_sintomas[serie_sintomas < 10]
    #data.drop(data[(data["Sintoma"].value_counts()<10)].index, inplace=True)
  #  vc = data["Sintoma"].value_counts()
    
   # data=data.drop(data["Sintoma"]==(vc[vc < 10].index))
    print ("Después de quitar registros duplicados y check nulos")
    print ("Enfermedades: ", data["Enfermedad"].nunique())
    print ("Sintomas: ", data["Sintoma"].nunique())
    print ("Frecuencias: ", data["Frecuencia"].nunique())
    vc = data["Sintoma"].value_counts()
    vector=vc[vc < 2].index
    for a in vector:
         # Get names of indexes for which column Stock has value No
        indexNames = data [ data["Sintoma"] == a ].index
        # Delete these row indexes from dataFrame
        for b in indexNames:
            data.drop(b , inplace=True, axis=0)
    print ("Después de borrado filas")
    print ("Enfermedades: ", data["Enfermedad"].nunique())
    print ("Sintomas: ", data["Sintoma"].nunique())
    print ("Frecuencias: ", data["Frecuencia"].nunique())  

    return data    

In [5]:
def prepara_df_train(data, corte):
    
    data2=data[(data['Frecuencia']=="Muy frecuente (99-80%)") | (data['Frecuencia']=="Frecuente (79-30%)")]
    if (corte!=0):
        data2 = data2.iloc[:corte]
    sintomas=data2.iloc[:,1]
    sintomas_sin_repe=sintomas.drop_duplicates()
    enfermedades=data2.iloc[:,0]
    enfermedades_sin_repe=enfermedades.drop_duplicates()
    df_train=pd.DataFrame(columns=sintomas_sin_repe)
    df_train.insert(0, 'Enfermedad', 0)
    df_train.insert(1, 'id_Enfermedad', 0)
    return enfermedades_sin_repe,sintomas_sin_repe, df_train, data2

In [6]:
def  datos_entreno(data,df_train, repeticiones,sintomas_sin_repe):
    data_agrupado = (data.groupby("Enfermedad")
         .agg({"Sintoma": np.array, "Frecuencia": np.array})
         .reset_index()
         )
    print (data_agrupado)
    z=0
    j=0
    while (z<repeticiones):
    
  #  print ("entra")
        i=0
        for a in data_agrupado["Enfermedad"]:
           # print ("Enfermedad: ", a)
        #vector_enfermedad.append(a)
        #lista=[]
            lst = [0] * ((len(sintomas_sin_repe)+2))
            df_train.loc[len(df_train)] = lst
            df_train["Enfermedad"][j]=a
            #df_train["id_Enfermedad"][j]=(i+1)
            df_train.loc[:, ('id_Enfermedad', j)] = (i+1)
            pos=0
            for b in data_agrupado["Sintoma"][i]:
                valor_aleatorio = random.random()
                frecuencia=data_agrupado["Frecuencia"][i][pos]
            #    print ("frecuencia:", frecuencia)
                if (frecuencia=="Muy frecuente (99-80%)"):
             #       print (b)
              #      print ("es muy frecuente")
                    if (valor_aleatorio>0.4):
                        valor_entero=1
                    else:
                        valor_entero=0
                elif (frecuencia=="Frecuente (79-30%)"):
               #     print (b)
                #    print ("es frecuente")
                    if (valor_aleatorio>0.6):
                        valor_entero=1
                    else:
                        valor_entero=0
                df_train[b][j]=valor_entero
                pos=pos+1
            j=j+1
            i=i+1
        z=z+1
        print ("VUELTA: ", z)
    return df_train        

In [7]:
def entrenar (df_train):
    target=df_train["id_Enfermedad"]
    df_train=df_train.drop (['Enfermedad'], axis=1)
    #X_train, X_test, y_train, y_test = train_test_split(df_train,target)
   # rf= RandomForestClassifier()
    #rf.fit (X_train,y_train)
    i
    prediccion=rf.predict(X_test)
    print (y_test, prediccion)
    return df_train,rf,target,X_test,y_test

In [ ]:
data=saca_info()

In [ ]:
enfermedades_sin_repe,sintomas_sin_repe, df_train,data_filtrado_frec=prepara_df_train(data,0)

In [ ]:
df_train

In [ ]:
sintomas_sin_repe

In [ ]:
enfermedades_sin_repe.info()
#ojo que perdemos enfermedades por duplicates sintomas impia

In [ ]:
data_filtrado_frec

In [ ]:
df_train=datos_entreno(data_filtrado_frec, df_train, 10, sintomas_sin_repe);

In [8]:
df_train.to_csv("df_train.csv", index=False)

NameError: name 'df_train' is not defined

In [9]:
df_train=pd.read_csv("df_train.csv")

In [10]:
df_train.shape

(42320, 5502)

In [ ]:
df_train,rf,target,X_test,y_test=entrenar (df_train)

In [ ]:
evaluate(rf,X_test,y_test)

In [ ]:
data_filtrado_frec

In [ ]:
predicciones = rf.predict_proba(X_test)
predicciones[:5, :]

In [ ]:
probs

In [ ]:
import numpy as np
df_predicciones = pd.DataFrame(data=predicciones, columns=['0', '1','2','3','4', '5', '6', '7', '8'])
df_predicciones['clasificacion_default_0.5'] = np.where(df_predicciones['0'] > df_predicciones['1'], 0, 1)
df_predicciones